# IMPORTS

In [8]:
import numpy as np
import pdb
import time

class color:
   BOLD = '\033[1m'
   END = '\033[0m'

# USER INPUTS

In [9]:
inp = int(input("Is the code for Report dataset(1) or Demo dataset(2): "))
file_path = input("Please enter the file name: ")
if(inp == 1):
    max_depth1 = int(input("Please enter Max Depth Value: ")) #100
    min_size1 = int(input("Please enter Min Size Value: ")) #10

Is the code for Report dataset(1) or Demo dataset(2): 1
Please enter the file name: data2.txt
Please enter Max Depth Value: 100
Please enter Min Size Value: 10


# FUNCTIONS

In [3]:
def featurecheck(val):
    try:
        float(val)
        return True
    except:
        return False

def get_fold_range(row_count,fold_count):
    k_main = []
    if(fold_count==1):
        step = round(0.23*row_count)
        temp = []
        temp.append(0)
        temp.append(step)
        k_main.append(temp)
    else:
        step = round(row_count / fold_count)
        rem = row_count - fold_count * step
        p = 0
        
        for i in range(fold_count):
            val_add = 0
            if(rem > 0):
                val_add = 1
            temp = []
            temp.append(p)
            p = p + step + val_add
            temp.append(p)
            k_main.append(temp)
            rem -= 1
        #print(k_main)
    return(k_main)


def preprocessing_dataset(file_path, inp):
    
    main_mapping_dict = {}
    nominal_features = []
    
    # Updating the nominal attributes by a number 
    with open(file_path, 'r') as file1:
        data1 = [line.strip().split('\t') for line in file1]
        temp_data = np.asarray(data1)
    
    (row_count, col_count) = temp_data.shape
    for i in range(col_count):
        if featurecheck(temp_data[0, i]) == False:
            nominal_features.append(i)
    
    for i in range(len(nominal_features)):
        temp_dict = {}
        unique_vals = np.unique(temp_data[:, nominal_features[i]]).tolist()
        
        temp1 = unique_vals
        for j in range(len(temp1)):
            temp_dict[temp1[j]] = j
        main_mapping_dict[i] = temp_dict

        for k in range(row_count):
            temp_data[k, nominal_features[i]] = temp_dict[temp_data[k, nominal_features[i]]]
    
    if(inp == 1):
        final_data = temp_data.astype(np.float)
        return final_data
    elif(inp == 2):
        final_data = temp_data.astype(np.int)
        return (final_data,main_mapping_dict)

def binary_feature_divide(data, feature_index, feature_val):
    left_list = []
    right_list = []
    for i in range(data.shape[0]):
        if data[i, feature_index] <= feature_val:
            left_list.append(i)
        else:
            right_list.append(i)

    left = data[left_list,:]
    right = data[right_list,:]
    return left, right

def calculate_gain(left, right):
    # SLIDE 52 - CLASSIFICATION 1
    C1_parent = list(left[:, -1]).count(0) + list(right[:, -1]).count(0)
    C2_parent = list(left[:, -1]).count(1) + list(right[:, -1]).count(1)
    
    root_gini = 1 - (C1_parent / (C1_parent + C2_parent))**2 - (C2_parent / (C1_parent + C2_parent))**2

    N1_left = list(left[:, -1]).count(0)
    N2_left = list(left[:, -1]).count(1)
    if (N1_left + N2_left) != 0:
        gini_left = 1 - (N1_left / (N1_left + N2_left))**2 - (N2_left / (N1_left + N2_left))**2
    else:
        gini_left = 0

    N1_right = list(right[:, -1]).count(0)
    N2_right = list(right[:, -1]).count(1)
    if (N1_right + N2_right) != 0:
        gini_right = 1 - (N1_right / (N1_right + N2_right))**2 - (N2_right / (N1_right + N2_right))**2
    else:
        gini_right = 0

    gini_children = ((N1_left + N2_left) / (C1_parent + C2_parent)) * gini_left + ((N1_right + N2_right) / (C1_parent + C2_parent)) * gini_right
    gain = root_gini - gini_children
    return gain

def get_best_feature_divide(feature_values):
    gain = 0
    dict1 = {}
    for row in range(feature_values.shape[0]):
        for col in range(feature_values.shape[1] - 1):
            left, right = binary_feature_divide(feature_values, col, feature_values[row, col])
            
            upd_gain = calculate_gain(left, right)
            if upd_gain > gain:
                feature_index = col
                feature_val = feature_values[row, col]
                gain = upd_gain
                dict1 = {"feature_index": feature_index, "feature_val": feature_val, "left": left, "right": right}
    #print("Gain = ", gain)
    return dict1

def terminal_node(node_values):
    if list(node_values[:, -1]).count(0) >= list(node_values[:, -1]).count(1):
        return 0
    else:
        return 1

def create_treebranch(full_tree_dict, max_depth, curr_depth, min_size):

    left = full_tree_dict["left"]
    right = full_tree_dict["right"]

    del(full_tree_dict["left"]) # or pop
    del(full_tree_dict["right"]) # or pop
    
    # No feature values left to iterate through so we get the terminal node 
    if left.shape[0] == 0:
        full_tree_dict["left"] = full_tree_dict["right"] = terminal_node(right)

    if right.shape[0] == 0:
        full_tree_dict["left"] = full_tree_dict["right"] = terminal_node(left)

    if curr_depth >= max_depth:
        full_tree_dict["left"] = terminal_node(left)
        full_tree_dict["right"] = terminal_node(right)

    if left.shape[0] <= min_size:
        full_tree_dict["left"] = terminal_node(left)
    else:
        if (len(np.unique(left[:, -1])) == 1):
            full_tree_dict["left"] = left[0, -1]
        else:
            full_tree_dict["left"] = create_treebranch(get_best_feature_divide(left), max_depth, curr_depth+1, min_size)

    if right.shape[0] <= min_size:
        full_tree_dict["right"] = terminal_node(right)
    else:
        if (len(np.unique(right[:, -1])) == 1):
            full_tree_dict["right"] = right[0, -1]
        else:
            full_tree_dict["right"] = create_treebranch(get_best_feature_divide(right), max_depth, curr_depth+1, min_size)

    return full_tree_dict

def find_testing_data_class(test_record, full_tree_dict):
    if test_record[full_tree_dict["feature_index"]] < full_tree_dict["feature_val"]:
        if type(full_tree_dict["left"]) != dict:
            return full_tree_dict["left"]
        else:
            return find_testing_data_class(test_record, full_tree_dict["left"])            
    else:
        if type(full_tree_dict["right"]) != dict:
            return full_tree_dict["right"]            
        else:
            return find_testing_data_class(test_record, full_tree_dict["right"])

def decision_tree(file_path,inp,max_depth,min_size):
    dataset = preprocessing_dataset(file_path, inp)

    tenfold_val = np.array_split(dataset, 10)

    #k_main = get_fold_range(dataset.shape[0],10)

    accuracy_list = []
    precision_list = []
    recall_list = []
    fmeasure_list = []
    
    for i in range(len(tenfold_val)):
        testing_data = tenfold_val[i]
        training_data = np.array([]).reshape(0, tenfold_val[0].shape[1])
        for j in range(len(tenfold_val)):
            if (i!=j):
                 training_data = np.concatenate((training_data, tenfold_val[j]), axis=0)
    
    #for i in k_main:
    #    
    #    training_data = dataset[i[1]:]
    #    testing_data = dataset[i[0]:i[1]]
    #    
        actual_label = []
        for j in testing_data:
            actual_label.append(j[-1])

        full_tree_dict = get_best_feature_divide(training_data)
        #max_depth = 100
        #min_size = 10
        curr_depth = 1
        full_tree_dict = create_treebranch(full_tree_dict, max_depth, curr_depth, min_size)

        prediction = []

        for test_record in testing_data:
            prediction.append(find_testing_data_class(test_record, full_tree_dict))

        (a,b,c,d) = performance_calculation(prediction, actual_label)

        accuracy = (a+d)/(a+b+c+d)
        precision = a/(a+c)
        recall = a/(a+b)
        fmeasure = (2*a)/((2*a)+b+c)
        accuracy_list.append(accuracy); precision_list.append(precision);
        recall_list.append(recall); fmeasure_list.append(fmeasure);
    
    return(accuracy_list, precision_list, recall_list, fmeasure_list)
        
def performance_calculation(prediction, actual_label):
    a = 0; b = 0; c = 0; d = 0;

    for i in range(len(prediction)):
        if actual_label[i] == 1 and prediction[i] == 1:
            a += 1
        elif actual_label[i] == 1 and prediction[i] == 0:
            b += 1
        elif actual_label[i] == 0 and prediction[i] == 1:
            c += 1
        elif actual_label[i] == 0 and prediction[i] == 0:
            d += 1

    return (a,b,c,d)

def display_measures(acc, prec, rec, fm):
    if(len(acc)==1):
        print("Algorithm Measures")
        print("Accuracy -> "+str(acc[0]))
        print("Precision -> "+str(prec[0]))
        print("Recall -> "+str(rec[0]))
        print("F-Measure -> "+str(fm[0]))
    else:
        avg_accuracy = 0; avg_precision = 0; avg_recall = 0; avg_fmeasure = 0;
        for i in range(len(acc)):
            print("----------------------------------------")
            print("Cross Validation Iteration -> "+str(i+1))
            print("Algorithm Measures")
            print("Accuracy -> "+str(acc[i]))
            print("Precision -> "+str(prec[i]))
            print("Recall -> "+str(rec[i]))
            print("F-Measure -> "+str(fm[i]))

            avg_accuracy += acc[i]; avg_precision += prec[i]; avg_recall += rec[i]; avg_fmeasure += fm[i];

        print("----------------------------------------")
        print("Average Measures")
        print("Average Accuracy -> "+str(avg_accuracy/len(acc)))
        print("Average Precision -> "+str(avg_precision/len(prec)))
        print("Average Recall -> "+str(avg_recall/len(rec)))
        print("Average F-Measure -> "+str(avg_fmeasure/len(fm)))

def display_demo_result(main_dict, full_tree_dict):
    for i in main_dict.keys():
        print("Feature Index (Column): "+str(i))
        #print("Feature Values: ")
        temp_dict = main_dict[i]
        for j in temp_dict.keys():
            print("Feature Value -> "+color.BOLD+str(j)+color.END+" is mapped to -> "+color.BOLD+str(temp_dict[j])+color.END)
        print()
    print(full_tree_dict)

# MAIN RUNNING OF CODE

In [4]:
if(inp==1):
    #file_path = input("Please enter the file name: ") #'data1.txt'
    #max_depth = int(input("Please enter Max Depth Value: "))
    #min_size = int(input("Please enter Min Size Value: "))
    (acc, prec, rec, fm) = decision_tree(file_path, inp, max_depth1, min_size1)
    display_measures(acc, prec, rec, fm)
else:
    #file_path = input("Please enter the file name: ") #"project3_dataset4.txt"
    (dataset,main_dict) = preprocessing_dataset(file_path, inp)

    full_tree_dict = get_best_feature_divide(dataset)
    max_depth = 100
    min_size = 1
    curr_depth = 1

    full_tree_dict = create_treebranch(full_tree_dict, max_depth, curr_depth, min_size)
    display_demo_result(main_dict, full_tree_dict)

----------------------------------------
Cross Validation Iteration -> 1
Algorithm Measures
Accuracy -> 0.9122807017543859
Precision -> 0.8461538461538461
Recall -> 0.9565217391304348
F-Measure -> 0.8979591836734694
----------------------------------------
Cross Validation Iteration -> 2
Algorithm Measures
Accuracy -> 0.9122807017543859
Precision -> 0.8
Recall -> 0.9411764705882353
F-Measure -> 0.8648648648648649
----------------------------------------
Cross Validation Iteration -> 3
Algorithm Measures
Accuracy -> 0.9824561403508771
Precision -> 1.0
Recall -> 0.9285714285714286
F-Measure -> 0.9629629629629629
----------------------------------------
Cross Validation Iteration -> 4
Algorithm Measures
Accuracy -> 0.9298245614035088
Precision -> 0.9473684210526315
Recall -> 0.8571428571428571
F-Measure -> 0.9
----------------------------------------
Cross Validation Iteration -> 5
Algorithm Measures
Accuracy -> 0.8771929824561403
Precision -> 0.8421052631578947
Recall -> 0.8
F-Measure ->

In [7]:
# https://stackoverflow.com/questions/10756427/loop-through-all-nested-dictionary-values
def myprint(d,assa):
    for k, v in d.items():
        if isinstance(v, dict):
            print(str(assa)+str(k))
            assa = str(assa)+"---"
            myprint(v,assa)
        else:
            print("{0}{1} : {2}".format(assa,k, v))
assa1 = "---" 
if(inp==2):
    myprint(full_tree_dict,assa1)